**Note:** This notebook is written in the Julia language, so the cells can't be
  executed in Google Colab. If you want to verify that the notebook works, I
  recommend [JuliaBox](https://juliabox.com/) or testing locally. The syntax is
  very similar to Python and MATLAB. Note in particular the dot syntax used to
  perform elementwise operations (`f.(x)` applies `f` to all elements of `x`),
  that indices start at 1 and that the last statement of all functions is returned
  automatically.

# **Lab 2: Direct methods**
**Anders Ågren Thuné**

# **Abstract**

In this report, a number of algorithms related to matrix inverses were
implemented. In particular, a householder QR factorization algorithm was
implemented and used to further implement a direct solver and a least squares
solver of systems of linear equations, and a function for calculating the
eigenvalues of a matrix using the QR eigenvalue algorithm. The algorithms worked
as expected without any major surprises.

# *About the code**

A short statement on who is the author of the file, and if the code is
distributed under a certain license.

In [1]:
"""
DD2363 Methods in Scientific Computing,
KTH Royal Institute of Technology, Stockholm, Sweden.
"""

# Copyright (C) 2019
# Anders Ågren Thuné (athune@kth.se)
# Johan Hoffman (jhoffman@kth.se)

# Code written by Anders Ågren Thuné based on the template by Johan Hoffman.

# This file is part of the course DD2363 Methods in Scientific Computing
# KTH Royal Institute of Technology, Stockholm, Sweden
#
# This is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

"DD2363 Methods in Scientific Computing,\nKTH Royal Institute of Technology, Stockholm, Sweden.\n"

# **Set up environment**

In [2]:
using LinearAlgebra
import Base: getproperty

# **Introduction**

Systems of linear equations appear frequently in a wide variety of problems. A
system of linear equations can be expressed in linear algebra terms as $Ax = b$,
which, if solvable, has the solution $x = A^{-1}b$. As such, being able to
compute the inverse of a given matrix is a problem of great importance. This is
difficult in general, but certain classes of matrices have easily computable
inverses; an orthogonal matrix $Q$ has the inverse $Q^T$ and the inverse of a
triangular matrix can be computed through back (or forward) substitution.
Therefore, a common approach to computing the inverse of a given matrix is to
factor it into triangular and orthogonal matrices, which can then be easily
inverted. One such factorization is the QR factorization in which any matrix $A$
is factored into an orthogonal matrix $Q$ and an upper triangular matrix $R$.
The inverse is then given by $A^{-1}=(QR)^{-1}=R^{-1}Q^T$.

Even when the system $Ax=b$ is unsolvable (there is no $A^{-1}), the best
possible solution can be obtained by projecting $b$ orthogonally onto
$range(A)$. This gives the approximated solution $\hat{x} = (A^TA)^{-1}A^Tb$,
where $(A^TA)^{-1}$ is called the *pseudo-inverse* of $A$.

The QR factorization can also be used in a wider range of problems. One such
problem is eigenvalue computation, where an iterative $QR$-algorithm can be used
to obtain a Schur factorization of a symmetric matrix. That is, a factorization
$A = QTQ^T$, where $T$ is a triangular matrix with the eigenvalues of $A$ on
the diagonal.

This report presents how these concepts, described in chapters 5 and 6 of the
lecture notes, were used to implement the following:
- A function for QR factorization
- A direct solver of of $Ax=b$
- A solver of the least squares problem $Ax=b$
- A function performing the QR eigenvalue algorithm

# **Methods**

## **QR factorization**

The QR factorization for a matrix can be calculated in a number of different
ways. The method implemented here is the *Householder QR factorization*, based
on Householder reflections. A matrix $P$ of the form $P = I-\beta vv^T, \quad
\beta = \frac{2}{v^Tv}$ is a *Householder reflection*. This is an orthogonal
reflector reflecting a given vector $x$ in the hyperplane
$\text{span}\{v\}^\perp. Selecting $v$ = $\pm \|x\|$ gives $Px = \pm\|x\|e_1$,
which can be utilized to construct $Q_n\dots Q_2Q_1A = R$, where each matrix
$Q_k$ is constructed to zero the subdiagonal elements of a column of $A$. This
is achieved by letting $Q_k = \begin{matrix} I & 0 \\ 0 P \end{matrix}$, where P
is a Householder reflection. When dealing with a Householder QR factorization,
it is advantageous not to explicitly form the matrices $Q$ or $Q_k$, but instead
utilize that $Q_kA = A - (\beta v)(v^TA)$ and $AQ_k = A - (A v)(\beta
v)^T$.

[Golub & Van Loan 2013: Chapter 5.1-5.2, Hoffman 2019: Chapter 5.2]

The following implementation is based on Algorithm 5.2.1 of Golub & Van Loan
(2013), and results in a matrix with the upper triangular part of $R$ as its
upper triangular part, and the vectors $v_j$ required to construct $Q$ below the
diagonal. To be precise, all elements except the first of these vectors are
stored, with the implicit assumption that they are normalized such that the
first element is always 1. In addition to the function itself, a wrapper
struct (analogous to a Python class) is implemented to facilitate easy handling
of this format.

In [3]:
"""
A struct for representing the QR factorization of a matrix
in a compact way. The constructor computes the factorization
using Algorithm 5.2.1 of Golub & Van Loan (2013),
modifying the argument in place.
"""
mutable struct QRfact{T <: AbstractFloat}
    QR :: Matrix{T}
    betas :: Vector{T}

    function QRfact(A :: Matrix{T}) where T <: AbstractFloat
        (m, n) = size(A)
        v = zeros(T, m)
        betas = zeros(T, n)

        for j = 1 : n
            v[j:m] .= A[j:m,j]
            v[j] += sign(v[j])*norm(v[j:m])
            betas[j] = 2v[j]^2/norm(v[j:m])^2
            v[j:m] ./= v[j]
            for k = j : n
                A[j:m,k] .-= betas[j].*v[j:m].*dot(v[j:m],A[j:m,k])
            end
            if j<m
               A[j+1:m,j] .= v[j+1:m]
            end
        end
        new{T}(A, betas)
    end
end

"""
Overloads getproperty to allow easy extraction of Q and R factors.
Enables dot syntax, as in the following example:
> QR = QRfactorization(A)
> Q = QR.q
> R = QR.r
"""
function getproperty(qr :: QRfact{T}, f :: Symbol) where T
    QR = getfield(qr, :QR)
    (m, n) = size(QR)
    if f == :q
        Q = Matrix{T}(I,m,m)
        mulQ!(qr,Q)
    elseif f == :qt
        Qt = Matrix{T}(I,m,m)
        mulQ!(qr,Qt,transpose=true)
    elseif f == :r
        triu(QR)
    elseif f == :factors
        QR
    else
        error("type $(typeof(qr)) has no field $f")
    end
end

"""
Multiply the matrix C with the Q factor of a QR factorization.
Keywords specify whether to multiply with Q or Q^T, and whether
to multiply from the left or from the right. Modifies C in place
and returns the result.
"""
function mulQ!(qr :: QRfact{T}, C :: VecOrMat{T};
               transpose::Bool = false, fromright::Bool = false) where T
    QR = getfield(qr, :QR)
    betas = getfield(qr, :betas)
    (m,n) = size(QR)
    Csize = typeof(C) <: Vector ? (length(C), 1) : size(C)

    index_to_match = Int(fromright)+1
    if Csize[index_to_match] != m
        throw(DomainError((m,Csize[index_to_match]),
            "The matrices' dimensions don't match!"))
    end
    iterateForwards = xor(fromright, transpose)
    iterRange = iterateForwards ? (1:n) : (n:-1:1)

    v = zeros(T, m)
    temp = zeros(T,Csize[1])
    for j = iterRange
        v[j] = one(T)
        v[j+1:m] .= QR[j+1:m,j]
        if fromright
            mul!(temp, C[:,j:m], v[j:m])
            for k = j : m
                C[:,k] .-= temp .*betas[j].*v[k]
            end
        else
            for k = 1 : Csize[2]
                C[j:m,k] .-= betas[j].*v[j:m].*dot(v[j:m],C[j:m,k])
            end
        end
    end
    C
end

"""
Get the QR factorization of A without modifying the original.
"""
QRfactorization(A :: Matrix{<:AbstractFloat}) = QRfact(copy(A))

QRfactorization

## **Direct solver**

The solution to the system $Ax=b$ can be obtained as $x = A^{-1}b = R^{-1}Q^Tb$,
once $A$ has been factored into $Q$ and $R$. Setting $y$ = $Q^Tb$, the equation
to solve is $x = R^{-1}y$. Algorithm 2 of chapter 5 in the notes describes how
such a system can be solved, using *back substitution*. In the implementation
below, the summation of the original algorithm is replaced by a dot product.

In [4]:
"""
Compute x = U^-1 b through back substitution for any upper triangular matrix U.
"""
function backsubstitute(U::Matrix, b::Vector)
    m = length(b)
    if m != size(U)[1]
        throw(DomainError((m,size(U)[1]), "The dimensions of U and b don't match!"))
    end
    x = zeros(m)
    x[m] = b[m]/U[m,m]
    for j = m-1:-1:1
        x[j] = (b[j] - dot(U[j,j+1:m],x[j+1:m]))/U[j,j]
    end
    x
end

backsubstitute

In [5]:
"""
Compute x = A^-1 b for any invertible matrix A.
"""
function directsolver(A::Matrix, b::Vector)
    qr = QRfactorization(A)
    y = copy(b)
    mulQ!(qr,y, transpose=true)
    backsubstitute(qr.factors,y)
end

directsolver

## **Least squares solver**

This is very similar to the above. In order to solve the least squares problem,
we solve $x = (A^TA)^{-1}(A^Tb)$, which is an equation solvable directly, as
$A^TA$ is a positive definite symmetric matrix for all $A$.

In [6]:
"""
Compute the best solution x to Ax = b for a non-invertible matrix A, given by
x = (A^T A)^-1 (A^T b)
"""
leastsquares(A::Matrix, b::Vector) = directsolver(A'A, A'b)

leastsquares

## **QR eigenvalue algorithm**

In order to find the eigenvalues of a matrix $A$, an iterative algorithm where
$A^{(k)}=R^{(k)}Q^{(k)}, \quad Q^{(k)}R^{(k)} = A^{(k-1)}$ can be used
(algorithm 6 of chapter 6 of the notes). This algorithm corresponds to
simultaneous power iteration, which if it converges results in $\displaystyle
\lim_{k->\infty}A^{(k)} = QTQ^T$, where $T$ is triangular with the eigenvalues
of $A$ on the diagonal. In particular, if $A$ is Hermitian or normal, $T$ is
diagonal, meaning the columns of $Q$ are eigenvectors of $A$.

In this implementation, it is assumed that the input matrix $A$ is real and
symmetric, such that all eigenvalues are real and that $T$ is diagonal. That
way, the square sum of the offdiagonal elements can be used as a suitable stopping
criterion for the iteration.

In [7]:
offdiagsquaresum(A::Matrix) = norm(A)^2-sum(A[i,i]^2 for i in 1:min(size(A)...))

function qr_evals(A::Matrix{T}, upperbound=10000) where T
    (m,n) = size(A)
    if m != n
        throw(DomainError((m,n),"Matrix not square!"))
    end
    temp = zeros(T,m,m)
    Q = Matrix{T}(I,m,m)
    A_k = copy(A)
    for k = 1:upperbound
        temp .= A_k
        qr_k = QRfact(temp)
        A_k .= qr_k.r
        mulQ!(qr_k, Q, fromright=true)
        mulQ!(qr_k, A_k, fromright=true)

        isapprox(offdiagsquaresum(A_k),0) && break
    end
    (diag(A_k), Q)
end

qr_evals (generic function with 2 methods)

# **Results**

This section presents tests verifying that the implemented functions work as
expected. Where possible, random data is used to verify the functions.

In [8]:
number_of_tests = 1000
dimension_range = (1:100,1:100)
value_range = 1:10000
tol = 1e-8

1.0e-8

## **QR factorization**

The following tests are performed:
- $R$ is upper triangular
- $\|Q^{T}Q-I\|_F \approx 0$
- $\|QR-A\|_F \approx 0$

In [9]:
function test_qrfac()
    m,n = -1,0
    for i = 1:number_of_tests
        while m < n # Don't test underdetermined systems
            (m,n) = rand.(dimension_range)
        end
        A = rand(m,n)*rand(value_range)
        QRf = QRfactorization(A)
        R = QRf.r
        Q = QRf.q
        @assert istriu(R)
        @assert isapprox(norm(Q'Q - I),0, atol = tol)
        @assert isapprox(norm(Q*R - A),0, atol = tol)
    end
    println("QR factorization tests cleared!")
end
test_qrfac()

QR factorization tests cleared!


## **Direct solver**

The following tests are performed:
- $\|Ax-b\| \approx 0$
- $\|x-y\| \approx 0$, where $y$ is a manufactured solution with $b = Ay$

In [10]:
function test_dirsolve()
    for i = 1:number_of_tests
        m = rand(dimension_range[1])
        A = rand(m,m)*rand(value_range)
        y = rand(m)
        b = A*y
        x = directsolver(A,b)
        @assert isapprox(norm(A*x - b),0, atol = tol)
        @assert isapprox(norm(x - y),0, atol = tol)
    end
    println("Direct solver tests cleared!")
end
test_dirsolve()

Direct solver tests cleared!


## **Least squares solver**

The following test is performed:
- $\|Ax-b\|$ is minimal

For this test, it proved hard to devise a random setup, given that the length of the
residual might vary depending on the system to be solved. Instead, a manual test was
performed for a problem with a known residual length. Then, a comparison on random
data against Julia's built in solver was performed.

Let $A = \begin{matrix} 1 & 2 \\ 1 & 1/2 \\ 1 & -1 \end{matrix}$ and $b =
\begin{matrix} 1/2 \\ 0 \\ 1 \end{matrix}$. We get $A^TA = \begin{matrix} 3 &
3/2 \\ 3/2 & 21/4 \end{matrix}$ and $A^Tb = \begin{matrix} 3/2 \\ 0
\end{matrix}$.

$(A^TA)^{-1} = \dots = \frac{1}{9}\begin{matrix} 7/2 & -1 \\ -1 & 2 \end{matrix}$,
so $(A^TA)^{-1}A^Tb = \begin{matrix} 7/12 \\ -1/3 \end{matrix}$.

In [11]:
function test_lsqrsolve()
    A = [1.0 2; 1 0.5; 1 -1]
    b = [0.5, 0, 1]
    best_sol = [7/12, -1/6]
    @assert isapprox(leastsquares(A,b), best_sol, atol = tol)
    for i = 1:number_of_tests
        n = rand(dimension_range[2])
        m = rand(n+1:n+50)
        A = rand(m,n)*rand(value_range)
        b = rand(m)
        @assert isapprox(leastsquares(A,b),A\b, atol = tol)
    end
    println("Least square solver tests cleared!")
end
test_lsqrsolve()

Least square solver tests cleared!


## **QR eigenvalue algorithm**

The following tests are performed:
- $det(A - \lambda_i I) \approx 0$
- $\|Av_i - \lambda_i v_i\| \approx 0$

In [12]:
function test_evals()
    for i = 1:number_of_tests
        m = 3
        A = rand(m,m)
        A .= [x>y ? A[x,y] : A[y,x] for x in 1:m, y in 1:m] # Make symmetric
        (evals, evectors) = qr_evals(A)
        for j = 1:m
            @assert isapprox(det(A-evals[j]*I),0, atol=tol*1000)
            @assert isapprox(norm(A*evectors[:,j].-evals[j]*evectors[:,j]),0, atol = tol*1000)
        end
    end
    println("Eigenvalue tests cleared!")
end
test_evals()

Eigenvalue tests cleared!


In order to get the above tests to pass, the dimension of the matrix had to be
limited to 4, the value range had to be restricted and the tolerance had to be
increased. Especially the determinant assertion was hard to pass, and if the
dimension was raised to more than 5, it would consistently be in the thousands
or higher.

In [13]:
function run_tests()
    test_qrfac()
    test_dirsolve()
    test_lsqrsolve()
    test_evals()
end
run_tests()

QR factorization tests cleared!


Direct solver tests cleared!


Least square solver tests cleared!


Eigenvalue tests cleared!


# **Discussion**

The algorithms worked as expected overall. However, one thing that was
unexpected was that the eigenvalue algorithm didn't pass the determinant test
for larger matrices. This was not because the eigenvalues themselves were
incorrect (which comparison with the built-in functions showed). Therefore, it
is likely that the algorithm used to compute the determinant of a matrix is very
unstable for matrices of large dimension, or that some major cancellation occurred
for the matrices that were tested.

# **References**

- Hoffman, J. 2019. *Introduction to Scientific Computing*
- Golub, Gene H. and Van Loan, Charles F. 2013. *Matrix Computations*. 4th ed. Baltimore: John Hopkins University Press.